# Препроцессинг

In [7]:
import re
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
class TwitterDataPreprocessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.final_df = None
        self.train_df = None
        self.val_df = None
        self.test_df = None

    def load_data(self, limit=100000):
        self.df = pd.read_csv(self.file_path).iloc[:limit].copy()
        self.df.columns = self.df.columns.str.strip()

        # Выбор нужных столбцов
        columns_needed = ['Weekday', 'Hour', 'Day', 'Reach', 'RetweetCount', 'Likes', 'text']
        self.df = self.df[columns_needed].copy()

    @staticmethod
    def clean_text(text):
        if not isinstance(text, str):
            return ""
        text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
        text = re.sub(r'\@\w+|\#', '', text)
        text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    @staticmethod
    def normalize_text(text):
        return " ".join(str(text).lower().split())

    @staticmethod
    def extract_keywords_per_tweet(text, top_n=5):
        words = str(text).split()
        common_words = Counter(words).most_common(top_n)
        return " ".join([word for word, _ in common_words])

    def preprocess(self):
        self.df['cleaned_text'] = self.df['text'].apply(self.clean_text)
        self.df['normalized_text'] = self.df['cleaned_text'].apply(self.normalize_text)
        self.df['keywords'] = self.df['normalized_text'].apply(self.extract_keywords_per_tweet)
        self.df['hashtags'] = self.df['text'].apply(lambda x: " ".join(re.findall(r"#\w+", str(x))))

        self.final_df = self.df[['Weekday', 'Hour', 'Day', 'Reach', 'RetweetCount', 'Likes', 'text',
                                 'cleaned_text', 'normalized_text', 'keywords', 'hashtags']]

    def split_and_save(self, output_dir="/content"):
        train_df, temp_df = train_test_split(self.final_df, test_size=0.3, random_state=42)
        val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Сохраняем файлы
        train_df.to_csv(f"{output_dir}/train.csv", index=False)
        val_df.to_csv(f"{output_dir}/val.csv", index=False)
        test_df.to_csv(f"{output_dir}/test.csv", index=False)

        print(f"Train size: {len(train_df)}")
        print(f"Validation size: {len(val_df)}")
        print(f"Test size: {len(test_df)}")

        return train_df, val_df, test_df

In [4]:
file_path = "/content/drive/MyDrive/Twitterdatainsheets.csv"
processor = TwitterDataPreprocessor(file_path)

processor.load_data()
processor.preprocess()
train_df, val_df, test_df = processor.split_and_save()
train_df.head()

<ipython-input-3-11f1d2ab7b11>:11: DtypeWarning: Columns (3,4,5,6,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.file_path).iloc[:limit].copy()


Train size: 70000
Validation size: 15000
Test size: 15000


,Weekday,Hour,Day,Reach,RetweetCount,Likes,text,cleaned_text,normalized_text,keywords,hashtags
76513,Monday,14,14,135.0,0.0,0.0,Netflix Backing Could Pump Up Google Cloud Vs....,Netflix Backing Could Pump Up Google Cloud Vs ...,netflix backing could pump up google cloud vs ...,netflix aws backing could pump,
60406,Thursday,18,25,15571.0,0.0,0.0,Incident response on the AWS cloud and the cas...,Incident response on the AWS cloud and the cas...,incident response on the aws cloud and the cas...,cloud the incident response on,#Cloud #Computing
27322,Friday,5,29,395.0,0.0,0.0,Aws Marketplace Channel Development Manager Ca...,Aws Marketplace Channel Development Manager Ca...,aws marketplace channel development manager ca...,channel seattle wa aws marketplace,#Seattle #WA
53699,Sunday,2,21,57.0,0.0,0.0,We are hiring: Senior Software Engineer - AWS ...,We are hiring Senior Software Engineer AWS Cod...,we are hiring senior software engineer aws cod...,senior software engineer aws we,#job
65412,Wednesday,12,2,106.0,0.0,0.0,New #Cloudstorage in S. Korea: @ZadaraStorage ...,New Cloudstorage in S Korea expands in AWS Seo...,new cloudstorage in s korea expands in aws seo...,in new cloudstorage s korea,#Cloudstorage #AWS


# Класс получение метрик из 3х параметров

In [5]:
class TweetMetricsScorer:
    def __init__(self, max_reach=None, max_retweet=None, max_likes=None):
        self.max_reach = max_reach
        self.max_retweet = max_retweet
        self.max_likes = max_likes

    def fit(self, df):
        """Автоматически определяет максимумы из датафрейма"""
        self.max_reach = self.max_reach or df['Reach'].max()
        self.max_retweet = self.max_retweet or df['RetweetCount'].max()
        self.max_likes = self.max_likes or df['Likes'].max()

    def score_row(self, reach, retweets, likes):
        # Логарифмическое преобразование (добавляем 1, чтобы избежать log(0))
        reach_score = np.log1p(reach)
        retweet_score = np.log1p(retweets)
        likes_score = np.log1p(likes)

        # Нормализация
        max_total = np.log1p(self.max_reach) * 0.4 + np.log1p(self.max_retweet) * 0.3 + np.log1p(self.max_likes) * 0.3
        weighted_score = (0.4 * reach_score + 0.3 * retweet_score + 0.3 * likes_score) / max_total

        return round(weighted_score * 10, 2)

    def add_score_column(self, df):
        self.fit(df)  # убедимся, что максимумы установлены
        df['score'] = df.apply(
            lambda row: self.score_row(row['Reach'], row['RetweetCount'], row['Likes']),
            axis=1
        )
        return df

In [37]:
scorer = TweetMetricsScorer()
scored_train_df = scorer.add_score_column(train_df)
scored_val_df = scorer.add_score_column(val_df)
scored_test_df = scorer.add_score_column(test_df)

# Сохраняем датафреймы в формате CSV
scored_train_df.to_csv('scored_train_data.csv', index=False)
scored_val_df.to_csv('scored_val_data.csv', index=False)
scored_test_df.to_csv('scored_test_data.csv', index=False)

# Выводим примеры данных
print(scored_train_df[['Reach', 'RetweetCount', 'Likes', 'score']].head())
print(scored_val_df[['Reach', 'RetweetCount', 'Likes', 'score']].head())
print(scored_test_df[['Reach', 'RetweetCount', 'Likes', 'score']].head())

         Reach  RetweetCount  Likes  score
76513    135.0           0.0    0.0   1.80
60406  15571.0           0.0    0.0   3.54
27322    395.0           0.0    0.0   2.19
53699     57.0           0.0    0.0   1.49
65412    106.0           0.0    0.0   1.71
         Reach  RetweetCount  Likes  score
3929   15545.0           0.0    0.0   3.54
66365     64.0           0.0    0.0   1.53
9267    1108.0           0.0    0.0   2.57
23724   1278.0           0.0    0.0   2.62
77309     79.0           0.0    0.0   1.61
          Reach  RetweetCount  Likes  score
81410      21.0           0.0    0.0   1.13
71182      48.0           0.0    0.0   1.43
44102      86.0           1.0    0.0   1.83
69451     939.0           0.0    0.0   2.51
78508  538527.0          12.0   20.0   6.38


# Тест

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sentence_transformers import SentenceTransformer

In [22]:
class TweetDataset(Dataset):
    def __init__(self, text_embeddings, numeric_features, scores):
        self.text_embeddings = text_embeddings
        self.numeric_features = numeric_features
        self.scores = scores

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        return (
            self.text_embeddings[idx],
            self.numeric_features[idx],
            self.scores[idx]
        )


class ViralityRegressor(nn.Module):
    def __init__(self, embedding_dim=384, numeric_dim=3, hidden_dim=512, dropout_rate=0.3):
        """
        Модель для предсказания метрики "виральности" от 0 до 10.

        embedding_dim: размерность эмбеддинга текста
        numeric_dim: количество числовых признаков (Weekday, Hour, Day)
        hidden_dim: размер скрытого слоя
        dropout_rate: вероятность dropout
        """
        super(ViralityRegressor, self).__init__()
        input_dim = embedding_dim + numeric_dim

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.norm = nn.LayerNorm(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)  # Одна метрика на выходе

    def forward(self, text_embedding, numeric_features):
        """
        text_embedding: тензор [batch_size, embedding_dim]
        numeric_features: тензор [batch_size, 3]
        """
        x = torch.cat([text_embedding, numeric_features], dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.norm(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.squeeze(1)  # [batch_size]


class TweetViralityPredictor:
    def __init__(self,
                 model_name="all-MiniLM-L6-v2",
                 embedding_dim=384,
                 hidden_dim=512,
                 dropout_rate=0.3,
                 learning_rate=0.001,
                 batch_size=64,
                 use_gpu=True):
        """
        Комплексный класс для предсказания виральности твитов.

        Параметры:
        model_name: Имя модели для SentenceTransformer
        embedding_dim: Размерность эмбеддингов
        hidden_dim: Размер скрытого слоя в модели
        dropout_rate: Вероятность dropout в модели
        learning_rate: Скорость обучения
        batch_size: Размер батча для обучения
        use_gpu: Использовать ли GPU, если доступно
        """
        self.device = torch.device("cuda" if use_gpu and torch.cuda.is_available() else "cpu")
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.model_name = model_name

        # Инициализация embedder
        self.embedder = SentenceTransformer(model_name)
        self.embedder.to(self.device)

        # Инициализация модели
        self.model = ViralityRegressor(embedding_dim=embedding_dim,
                                       numeric_dim=3,
                                       hidden_dim=hidden_dim,
                                       dropout_rate=dropout_rate)
        self.model.to(self.device)

        # Инициализация оптимизатора и планировщика скорости обучения
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.lr_scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.5)
        self.criterion = nn.MSELoss()

        # Словарь для маппинга категориальных признаков
        self.weekday_map = None
        self.hour_map = None
        self.day_map = None

    def _prepare_data(self, df, prediction_mode=False):
        """
        Подготовка данных: получение эмбеддингов и числовых признаков

        Параметры:
        df: DataFrame с данными
        prediction_mode: Если True, не требуется колонка 'score'
        """
        # Проверим колонку score только если не в режиме предсказания
        if 'score' not in df.columns and not prediction_mode:
            print("Нет колонки 'score' в DataFrame")

        # Создадим маппинг для категориальных признаков, если еще не создан
        if self.weekday_map is None:
            self.weekday_map = {day: i for i, day in enumerate(df['Weekday'].unique())}
            self.hour_map = {hour: i for i, hour in enumerate(df['Hour'].unique())}
            self.day_map = {day: i for i, day in enumerate(df['Day'].unique())}

        # Преобразуем категориальные признаки в числовые
        weekday_numeric = df['Weekday'].map(self.weekday_map).values
        hour_numeric = df['Hour'].map(self.hour_map).values
        day_numeric = df['Day'].map(self.day_map).values

        # Объединим числовые признаки
        numeric_features = np.column_stack([weekday_numeric, hour_numeric, day_numeric])
        numeric_features = torch.tensor(numeric_features, dtype=torch.float32)

        # Получаем эмбеддинги для текстов
        print("Получение эмбеддингов...")
        texts = df['normalized_text'].tolist()

        # Используем батчи для обработки больших датасетов
        embeddings = []
        batch_size = 128  # можно настроить в зависимости от доступной памяти
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            batch_embeddings = self.embedder.encode(batch_texts, convert_to_tensor=True)
            embeddings.append(batch_embeddings.cpu())

        text_embeddings = torch.cat(embeddings, dim=0)

        # Получаем целевую переменную (score), если не в режиме предсказания
        if prediction_mode:
            # При предсказании используем нулевые значения как заглушку
            scores = torch.zeros(len(df), dtype=torch.float32)
        else:
            scores = torch.tensor(df['score'].values, dtype=torch.float32)

        return text_embeddings, numeric_features, scores

    def train(self, train_df, val_df=None, num_epochs=20):
        """
        Обучение модели на подготовленных данных.

        Параметры:
        train_df: DataFrame с обучающей выборкой
        val_df: DataFrame с валидационной выборкой (опционально)
        num_epochs: Количество эпох обучения
        """
        print(f"Подготовка обучающих данных...")
        train_embeddings, train_numeric, train_scores = self._prepare_data(train_df)

        train_dataset = TweetDataset(train_embeddings, train_numeric, train_scores)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        if val_df is not None:
            print(f"Подготовка валидационных данных...")
            val_embeddings, val_numeric, val_scores = self._prepare_data(val_df)
            val_dataset = TweetDataset(val_embeddings, val_numeric, val_scores)
            val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        # Обучение модели
        print(f"Обучение модели на {len(train_df)} примерах...")
        self.model.to(self.device)
        for epoch in range(num_epochs):
            self.model.train()
            epoch_loss = 0.0

            for batch_idx, (text_embed, numeric_feats, targets) in enumerate(tqdm(train_loader)):
                text_embed = text_embed.to(self.device)
                numeric_feats = numeric_feats.to(self.device)
                targets = targets.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(text_embed, numeric_feats)
                loss = self.criterion(outputs, targets)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

            self.lr_scheduler.step()
            avg_loss = epoch_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

            # Оценка на валидационной выборке
            if val_df is not None:
                val_metrics = self.evaluate(val_loader)
                print(f"Validation - MAE: {val_metrics['mae']:.4f}, MSE: {val_metrics['mse']:.4f}")

        print("Обучение завершено!")

    def evaluate(self, val_loader=None, test_df=None):
        """
        Оценка производительности модели на валидационных или тестовых данных.

        Параметры:
        val_loader: DataLoader с валидационной выборкой (опционально)
        test_df: DataFrame с тестовой выборкой (опционально)

        Возвращает:
        Словарь с метриками MAE и MSE
        """
        if test_df is not None:
            # Подготовим тестовые данные, если они предоставлены в виде DataFrame
            test_embeddings, test_numeric, test_scores = self._prepare_data(test_df)
            test_dataset = TweetDataset(test_embeddings, test_numeric, test_scores)
            val_loader = DataLoader(test_dataset, batch_size=self.batch_size)

        if val_loader is None:
            return {"mae": None, "mse": None}

        self.model.eval()
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for text_embed, numeric_feats, targets in val_loader:
                text_embed = text_embed.to(self.device)
                numeric_feats = numeric_feats.to(self.device)

                outputs = self.model(text_embed, numeric_feats)
                all_preds.extend(outputs.cpu().numpy())
                all_targets.extend(targets.numpy())

        mae = mean_absolute_error(all_targets, all_preds)
        mse = mean_squared_error(all_targets, all_preds)

        return {"mae": mae, "mse": mse}

    def predict(self, tweets_df):
        """
        Предсказание виральности для новых твитов.

        Параметры:
        tweets_df: DataFrame с твитами для предсказания. Должен содержать колонки 'normalized_text', 'Weekday', 'Hour', 'Day'

        Возвращает:
        DataFrame с предсказанными значениями score
        """
        # Проверим, что все необходимые колонки присутствуют
        required_cols = ['normalized_text', 'Weekday', 'Hour', 'Day']
        for col in required_cols:
            if col not in tweets_df.columns:
                raise ValueError(f"Колонка {col} отсутствует в датафрейме")

        # Подготовим данные для предсказания с флагом prediction_mode=True
        text_embeddings, numeric_features, dummy_scores = self._prepare_data(tweets_df, prediction_mode=True)

        # Создадим датасет и DataLoader
        pred_dataset = TweetDataset(text_embeddings, numeric_features, dummy_scores)
        pred_loader = DataLoader(pred_dataset, batch_size=self.batch_size)

        # Предсказание
        self.model.eval()
        all_preds = []

        with torch.no_grad():
            for text_embed, numeric_feats, _ in pred_loader:
                text_embed = text_embed.to(self.device)
                numeric_feats = numeric_feats.to(self.device)

                outputs = self.model(text_embed, numeric_feats)
                all_preds.extend(outputs.cpu().numpy())

        # Добавим предсказания в датафрейм
        result_df = tweets_df.copy()
        result_df['predicted_score'] = all_preds

        return result_df

    def save_model(self, path="tweet_virality_model.pt"):
        """Сохранение модели и маппингов"""
        model_state = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'lr_scheduler_state_dict': self.lr_scheduler.state_dict(),
            'weekday_map': self.weekday_map,
            'hour_map': self.hour_map,
            'day_map': self.day_map,
            'model_name': self.model_name,
            'embedding_dim': self.embedding_dim
        }
        torch.save(model_state, path)
        print(f"Модель сохранена в {path}")

    def load_model(self, path="tweet_virality_model.pt"):
        """Загрузка модели и маппингов"""
        model_state = torch.load(path, map_location=self.device)

        # Обновим атрибуты
        self.model_name = model_state.get('model_name', self.model_name)
        self.embedding_dim = model_state.get('embedding_dim', self.embedding_dim)

        # Загрузим маппинги
        self.weekday_map = model_state['weekday_map']
        self.hour_map = model_state['hour_map']
        self.day_map = model_state['day_map']

        # Загрузим состояние модели и оптимизатора
        self.model.load_state_dict(model_state['model_state_dict'])
        self.optimizer.load_state_dict(model_state['optimizer_state_dict'])
        self.lr_scheduler.load_state_dict(model_state['lr_scheduler_state_dict'])

        print(f"Модель загружена из {path}")

# Обучение

In [66]:
# Пример использования класса
predictor = TweetViralityPredictor(use_gpu=True)

# Обучение модели
predictor.train(train_df, val_df, num_epochs=10)

# Оценка на тестовом наборе
test_metrics = predictor.evaluate(test_df=test_df)
print(f"Test MAE: {test_metrics['mae']:.4f}, MSE: {test_metrics['mse']:.4f}")

# Сохранение модели
predictor.save_model("tweet_virality_model1.pt")


Подготовка обучающих данных...
Получение эмбеддингов...


100%|██████████| 547/547 [15:58<00:00,  1.75s/it]


Подготовка валидационных данных...
Получение эмбеддингов...


100%|██████████| 118/118 [03:25<00:00,  1.74s/it]


Обучение модели на 70000 примерах...


...

Epoch 9/10, Loss: 0.5705
Validation - MAE: 0.5637, MSE: 0.5941


100%|██████████| 1094/1094 [00:08<00:00, 130.07it/s]


Epoch 10/10, Loss: 0.5640
Validation - MAE: 0.5691, MSE: 0.5985
Обучение завершено!
Получение эмбеддингов...


100%|██████████| 118/118 [03:22<00:00,  1.72s/it]


Test MAE: 0.5644, MSE: 0.5902
Модель сохранена в tweet_virality_model1.pt


# Предикт

In [ ]:
tweet_data = {
    "text": "Amazon Web Services Launches Korean Datacenters for Its Cloud Computing Platform - Business ... http://www.businesswire.com/news/home/20160106006789/en/Amazon-Web-Services-Launches-Korean-Datacenters-Cloud #Cloud #Computing",
    "Weekday": "Wednesday",
    "Hour": 20,
    "Day": 6
}

In [ ]:
# Функции для обработки текста
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_text(text):
    return " ".join(str(text).lower().split())

# Создаем DataFrame
day = tweet_data.get('Day', 1)

new_tweets_df = pd.DataFrame([{
    'text': tweet_data['text'],
    'normalized_text': normalize_text(clean_text(tweet_data['text'])),
    'Weekday': tweet_data['Weekday'],
    'Hour': tweet_data['Hour'],
    'Day': day
}])


In [34]:
# Загрузка модели
new_predictor = TweetViralityPredictor(use_gpu=True)
new_predictor.load_model("tweet_virality_model.pt")

# Предсказание для новых данных
new_tweets_df = new_tweets_df
results = new_predictor.predict(new_tweets_df)
print(results.head())

Модель загружена из tweet_virality_model.pt
Получение эмбеддингов...


100%|██████████| 1/1 [00:00<00:00, 35.36it/s]

                                                text  \
0  Amazon Web Services Launches Korean Datacenter...   

                                     normalized_text    Weekday  Hour  Day  \
0  amazon web services launches korean datacenter...  Wednesday    20    6   

   predicted_score  
0          2.44172  


# Улучшение Твитов И Рекомендации

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
from collections import Counter
import torch
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
# Убедитесь, что NLTK ресурсы загружены
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

class TweetEnhancer:
    def __init__(self, historical_df, virality_predictor=None, high_threshold=7.0):
        """
        Инициализация класса для улучшения твитов и выдачи рекомендаций.

        Параметры:
        historical_df: DataFrame с историческими твитами для анализа.
                      Должен содержать колонки 'normalized_text', 'text', 'score', 'Weekday', 'Hour'
        virality_predictor: Объект TweetViralityPredictor для предсказания виральности
        high_threshold: Порог для определения высокоэффективных твитов (score >= high_threshold)
        """
        self.df = historical_df
        self.virality_predictor = virality_predictor
        self.high_threshold = high_threshold

        # Создаем DataFrame с высокоэффективными твитами
        self.high_performing_tweets = self.df[self.df['score'] >= high_threshold]

        # Извлекаем полезные паттерны
        self._extract_patterns()

        # Находим оптимальное время для публикаций
        self._analyze_optimal_posting_times()

        # Извлекаем полезные хэштеги
        self._extract_hashtags()

        # Создаем TF-IDF векторайзер для сравнения текстов
        self._create_tfidf_vectorizer()

    def _extract_patterns(self):
        """Извлечение полезных паттернов из высокоэффективных твитов"""
        # Паттерны для call-to-action (CTA)
        cta_patterns = [
            r"(?i)(retweet|rt|share)(\s+if|\s+to|\s+when|\s+for)",
            r"(?i)(like|fav|favorite)(\s+if|\s+to|\s+when|\s+for)",
            r"(?i)(follow|subscribe)(\s+if|\s+to|\s+when|\s+for)",
            r"(?i)(tag|mention)(\s+someone|\s+a friend|\s+who)",
            r"(?i)(reply|respond|comment)(\s+with|\s+if|\s+to)",
            r"(?i)(what do you think|your thoughts|agree\?)",
            r"(?i)(don't forget to|remember to|be sure to)",
            r"(?i)(click|tap|swipe|check out)",
            r"(?i)(join|participate|take part|vote)",
            r"(?i)(read more|learn more|find out more)"
        ]

        # Словарь для хранения найденных CTA и их эффективности
        self.cta_effectiveness = {}

        for pattern in cta_patterns:
            # Находим твиты с данным паттерном
            mask = self.df['text'].str.contains(pattern, regex=True, na=False)
            tweets_with_pattern = self.df[mask]

            if len(tweets_with_pattern) > 0:
                # Среднее значение score для твитов с этим паттерном
                avg_score = tweets_with_pattern['score'].mean()
                # Количество высокоэффективных твитов с этим паттерном
                high_perf_count = len(tweets_with_pattern[tweets_with_pattern['score'] >= self.high_threshold])

                # Пример твита с данным паттерном и высоким score
                examples = []
                if high_perf_count > 0:
                    high_perf_examples = tweets_with_pattern[tweets_with_pattern['score'] >= self.high_threshold]
                    for _, row in high_perf_examples.head(2).iterrows():
                        # Извлекаем сам CTA из твита
                        text = row['text']
                        match = re.search(pattern, text, re.IGNORECASE)
                        if match:
                            cta_text = text[match.start():match.end()]
                            examples.append((cta_text, row['score']))

                # Сохраняем информацию о паттерне
                self.cta_effectiveness[pattern] = {
                    'avg_score': avg_score,
                    'high_perf_count': high_perf_count,
                    'total_count': len(tweets_with_pattern),
                    'examples': examples
                }

        # Извлечение эффективных фраз (n-граммы)
        self.effective_phrases = self._extract_ngrams()

    def _extract_ngrams(self, max_n=3, min_count=3):
        """
        Извлечение эффективных n-грамм из высокоэффективных твитов

        Параметры:
        max_n: Максимальная длина n-граммы
        min_count: Минимальное количество вхождений для учета n-граммы
        """
        stop_words = set(stopwords.words('english'))
        effective_ngrams = {}

        # Токенизация твитов
        high_perf_tokens = []
        for text in self.high_performing_tweets['normalized_text']:
            tokens = [t.lower() for t in word_tokenize(text) if t.isalpha() and t.lower() not in stop_words]
            high_perf_tokens.append(tokens)

        # Извлечение n-грамм для разных n
        for n in range(1, max_n + 1):
            ngram_counter = Counter()

            for tokens in high_perf_tokens:
                token_ngrams = list(ngrams(tokens, n))
                ngram_counter.update(token_ngrams)

            # Отбор часто встречающихся n-грамм
            effective_ngrams[n] = {
                ng: count for ng, count in ngram_counter.items()
                if count >= min_count
            }

        return effective_ngrams

    def _analyze_optimal_posting_times(self):
        """Анализ оптимального времени для публикации твитов"""
        # Группировка по дню недели и часу с подсчетом средней виральности
        time_analysis = self.df.groupby(['Weekday', 'Hour'])['score'].agg(['mean', 'count']).reset_index()

        # Отфильтруем только временные слоты с достаточным количеством твитов (хотя бы 5)
        time_analysis = time_analysis[time_analysis['count'] >= 5]

        # Отсортируем по убыванию среднего score
        self.optimal_posting_times = time_analysis.sort_values(by='mean', ascending=False)

    def _extract_hashtags(self):
        """Извлечение эффективных хэштегов из твитов"""
        # Регулярное выражение для извлечения хэштегов
        hashtag_pattern = r'#(\w+)'

        # Словарь для хранения статистики по хэштегам
        hashtag_stats = {}

        # Извлечение хэштегов из всех твитов
        for _, row in self.df.iterrows():
            text = row['text']
            score = row['score']

            hashtags = re.findall(hashtag_pattern, text)
            for hashtag in hashtags:
                hashtag = hashtag.lower()
                if hashtag not in hashtag_stats:
                    hashtag_stats[hashtag] = {
                        'total_count': 0,
                        'high_perf_count': 0,
                        'total_score': 0
                    }

                hashtag_stats[hashtag]['total_count'] += 1
                hashtag_stats[hashtag]['total_score'] += score

                if score >= self.high_threshold:
                    hashtag_stats[hashtag]['high_perf_count'] += 1

        # Вычисление среднего score для каждого хэштега
        for hashtag in hashtag_stats:
            if hashtag_stats[hashtag]['total_count'] > 0:
                hashtag_stats[hashtag]['avg_score'] = hashtag_stats[hashtag]['total_score'] / hashtag_stats[hashtag]['total_count']
            else:
                hashtag_stats[hashtag]['avg_score'] = 0

        # Сохраняем только хэштеги, встречающиеся хотя бы 3 раза
        self.hashtag_stats = {h: stats for h, stats in hashtag_stats.items() if stats['total_count'] >= 3}

        # Сортировка хэштегов по среднему score
        self.top_hashtags = sorted(
            self.hashtag_stats.items(),
            key=lambda x: x[1]['avg_score'],
            reverse=True
        )

    def _create_tfidf_vectorizer(self):
        """Создание TF-IDF векторайзера для сравнения текстов"""
        # Обучаем векторайзер на всех текстах
        self.tfidf_vectorizer = TfidfVectorizer(
            max_features=5000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.df['normalized_text'])

    def enhance_tweet(self, tweet_text, include_time_recommendation=True):
        """
        Улучшение твита и предоставление рекомендаций.

        Параметры:
        tweet_text: Текст твита для улучшения
        include_time_recommendation: Включать ли рекомендации по времени публикации

        Возвращает:
        Словарь с рекомендациями
        """
        # Нормализуем текст так же, как это делается для обучения модели
        normalized_text = tweet_text.lower()

        # Создаем пустой датафрейм для прогнозирования виральности
        current_time = datetime.now()
        weekday = current_time.strftime('%A')
        hour = current_time.hour
        day = current_time.day

        tweet_df = pd.DataFrame({
            'text': [tweet_text],
            'normalized_text': [normalized_text],
            'Weekday': [weekday],
            'Hour': [hour],
            'Day': [day]
        })

        # Создаем рекомендации
        recommendations = {
            'original_tweet': tweet_text,
            'suggestions': {
                'phrasing': self._suggest_phrasing(normalized_text),
                'hashtags': self._suggest_hashtags(normalized_text),
                'cta': self._suggest_cta(normalized_text)
            }
        }

        # Предсказание виральности исходного твита
        if self.virality_predictor:
            predicted = self.virality_predictor.predict(tweet_df)
            original_score = predicted['predicted_score'].iloc[0]
            recommendations['original_score'] = original_score

            # Предсказание виральности улучшенного твита
            enhanced_tweet = self._create_enhanced_tweet(tweet_text, recommendations)
            enhanced_df = pd.DataFrame({
                'text': [enhanced_tweet],
                'normalized_text': [enhanced_tweet.lower()],
                'Weekday': [weekday],
                'Hour': [hour],
                'Day': [day]
            })

            enhanced_predicted = self.virality_predictor.predict(enhanced_df)
            enhanced_score = enhanced_predicted['predicted_score'].iloc[0]
            recommendations['enhanced_score'] = enhanced_score
            recommendations['enhanced_tweet'] = enhanced_tweet
            recommendations['improvement'] = enhanced_score - original_score

        # Добавляем рекомендации по времени публикации, если требуется
        if include_time_recommendation:
            recommendations['posting_time'] = self._recommend_posting_time()

        return recommendations

    def _suggest_phrasing(self, text, num_suggestions=3):
        """
        Предложение улучшений формулировки на основе эффективных фраз.

        Параметры:
        text: Текст твита
        num_suggestions: Количество предлагаемых улучшений

        Возвращает:
        Список предложений по улучшению формулировки
        """
        suggestions = []

        # Поиск похожих эффективных твитов
        text_vector = self.tfidf_vectorizer.transform([text])
        similarity_scores = cosine_similarity(text_vector, self.tfidf_matrix).flatten()

        # Индексы топ-5 наиболее похожих твитов
        most_similar_indices = similarity_scores.argsort()[-5:][::-1]
        similar_high_perf_tweets = []

        for idx in most_similar_indices:
            if self.df.iloc[idx]['score'] >= self.high_threshold:
                similar_high_perf_tweets.append(self.df.iloc[idx]['text'])

        # Если нашли похожие высокоэффективные твиты, добавляем их как примеры
        if similar_high_perf_tweets:
            suggestions.append({
                'type': 'example',
                'description': 'Похожие высокоэффективные твиты для вдохновения:',
                'examples': similar_high_perf_tweets[:2]
            })

        # Проверка наличия эффективных фраз в тексте
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [t for t in tokens if t.isalpha() and t.lower() not in stop_words]

        # Проверяем, можно ли добавить эффективные биграммы или триграммы
        effective_bigrams = list(self.effective_phrases.get(2, {}).keys())
        effective_trigrams = list(self.effective_phrases.get(3, {}).keys())

        if effective_bigrams:
            # Сортируем по частоте
            effective_bigrams.sort(key=lambda x: self.effective_phrases[2][x], reverse=True)
            suggestions.append({
                'type': 'phrase',
                'description': 'Попробуйте добавить эти эффективные фразы:',
                'phrases': [' '.join(bigram) for bigram in effective_bigrams[:3]]
            })

        # Если текст короткий, предложим расширить
        if len(filtered_tokens) < 10:
            suggestions.append({
                'type': 'length',
                'description': 'Твит слишком короткий. Попробуйте расширить его, добавив больше контекста или деталей.'
            })
        # Если текст слишком длинный, предложим сократить
        elif len(filtered_tokens) > 30:
            suggestions.append({
                'type': 'length',
                'description': 'Твит слишком длинный. Попробуйте сократить его для лучшего восприятия.'
            })

        return suggestions

    def _suggest_hashtags(self, text, max_suggestions=5):
        """
        Предложение хэштегов на основе анализа текста и эффективных хэштегов.

        Параметры:
        text: Текст твита
        max_suggestions: Максимальное количество предлагаемых хэштегов

        Возвращает:
        Список рекомендуемых хэштегов
        """
        # Извлечение уже используемых хэштегов
        existing_hashtags = set(re.findall(r'#(\w+)', text.lower()))

        # Токенизация для поиска ключевых слов
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        keywords = [t for t in tokens if t.isalpha() and t.lower() not in stop_words]

        # Поиск тематически связанных хэштегов
        related_hashtags = []

        for hashtag, stats in self.hashtag_stats.items():
            # Пропускаем уже использованные хэштеги
            if hashtag in existing_hashtags:
                continue

            # Проверяем, связан ли хэштег с ключевыми словами в тексте
            if hashtag in keywords or any(hashtag in keyword for keyword in keywords):
                related_hashtags.append((hashtag, stats['avg_score']))

        # Сортируем по среднему score
        related_hashtags.sort(key=lambda x: x[1], reverse=True)

        # Если мало тематически связанных хэштегов, добавляем топовые хэштеги
        if len(related_hashtags) < max_suggestions:
            top_hashtags = [
                (hashtag, stats['avg_score'])
                for hashtag, stats in self.top_hashtags
                if hashtag not in existing_hashtags and not any(hashtag == rh[0] for rh in related_hashtags)
            ]

            # Объединяем списки и снова сортируем
            all_hashtags = related_hashtags + top_hashtags
            all_hashtags.sort(key=lambda x: x[1], reverse=True)

            # Выбираем top max_suggestions хэштегов
            suggested_hashtags = [f"#{h[0]}" for h in all_hashtags[:max_suggestions]]
        else:
            suggested_hashtags = [f"#{h[0]}" for h in related_hashtags[:max_suggestions]]

        return suggested_hashtags

    def _suggest_cta(self, text):
        """
        Предложение улучшений призывов к действию (CTA).

        Параметры:
        text: Текст твита

        Возвращает:
        Список предложений по улучшению CTA
        """
        # Проверяем, есть ли уже CTA в твите
        has_cta = any(re.search(pattern, text, re.IGNORECASE) for pattern in self.cta_effectiveness)

        suggestions = []

        if not has_cta:
            # Сортируем CTA по эффективности
            effective_ctas = sorted(
                self.cta_effectiveness.items(),
                key=lambda x: (x[1]['high_perf_count'] / max(1, x[1]['total_count']), x[1]['avg_score']),
                reverse=True
            )

            # Выбираем топ-3 эффективных CTA с примерами
            top_ctas = []
            for pattern, stats in effective_ctas[:3]:
                if stats['examples']:
                    example_text, example_score = stats['examples'][0]
                    top_ctas.append({
                        'cta_type': self._get_cta_type(pattern),
                        'example': example_text,
                        'score': example_score
                    })

            if top_ctas:
                suggestions.append({
                    'type': 'cta',
                    'description': 'Добавьте призыв к действию (CTA) для увеличения вовлеченности:',
                    'examples': top_ctas
                })
        else:
            suggestions.append({
                'type': 'cta',
                'description': 'В твите уже есть призыв к действию, что хорошо для вовлеченности.'
            })

        return suggestions

    def _get_cta_type(self, pattern):
        """Определение типа CTA по регулярному выражению"""
        if 'retweet' in pattern.lower() or 'rt' in pattern.lower() or 'share' in pattern.lower():
            return 'Ретвит/Репост'
        elif 'like' in pattern.lower() or 'fav' in pattern.lower():
            return 'Лайк'
        elif 'follow' in pattern.lower() or 'subscribe' in pattern.lower():
            return 'Подписка'
        elif 'tag' in pattern.lower() or 'mention' in pattern.lower():
            return 'Упоминание'
        elif 'reply' in pattern.lower() or 'respond' in pattern.lower() or 'comment' in pattern.lower():
            return 'Комментарий'
        elif 'think' in pattern.lower() or 'thoughts' in pattern.lower() or 'agree' in pattern.lower():
            return 'Вопрос/Опрос'
        else:
            return 'Общий призыв'

    def _recommend_posting_time(self, top_n=3):
        """
        Рекомендация оптимального времени для публикации.

        Параметры:
        top_n: Количество рекомендуемых временных слотов

        Возвращает:
        Список рекомендуемых временных слотов с метриками
        """
        # Берем топ-N временных слотов
        top_times = self.optimal_posting_times.head(top_n)

        recommended_times = []
        for _, row in top_times.iterrows():
            recommended_times.append({
                'weekday': row['Weekday'],
                'hour': row['Hour'],
                'avg_score': row['mean'],
                'sample_size': row['count']
            })

        return recommended_times

    def _create_enhanced_tweet(self, original_tweet, recommendations):
        """
        Создание улучшенной версии твита на основе рекомендаций.

        Параметры:
        original_tweet: Исходный текст твита
        recommendations: Словарь с рекомендациями

        Возвращает:
        Улучшенный текст твита
        """
        enhanced_tweet = original_tweet

        # Удаляем существующие хэштеги для замены на рекомендованные
        enhanced_tweet = re.sub(r'#\w+', '', enhanced_tweet).strip()

        # Добавляем рекомендованные хэштеги
        if recommendations['suggestions']['hashtags']:
            hashtag_text = ' ' + ' '.join(recommendations['suggestions']['hashtags'][:3])
            enhanced_tweet += hashtag_text

        # Добавляем CTA, если его нет
        cta_suggestions = recommendations['suggestions']['cta']
        has_cta = any(suggestion['description'].startswith('В твите уже есть призыв') for suggestion in cta_suggestions)

        if not has_cta and len(cta_suggestions) > 0 and 'examples' in cta_suggestions[0]:
            # Добавляем первый пример CTA
            example = cta_suggestions[0]['examples'][0]['example']
            if not re.search(re.escape(example), enhanced_tweet, re.IGNORECASE):
                enhanced_tweet += f" {example}"

        return enhanced_tweet.strip()

    def get_content_insights(self):
        """
        Получение обобщенных инсайтов о контенте.

        Возвращает:
        Словарь с инсайтами о контенте
        """
        insights = {
            'top_posting_times': self._recommend_posting_time(5),
            'top_hashtags': [{'hashtag': h, 'avg_score': stats['avg_score']}
                             for h, stats in self.top_hashtags[:10]],
            'effective_ctas': [],
            'content_patterns': []
        }

        # Топ-5 эффективных CTA
        effective_ctas = sorted(
            self.cta_effectiveness.items(),
            key=lambda x: (x[1]['high_perf_count'] / max(1, x[1]['total_count']), x[1]['avg_score']),
            reverse=True
        )

        for pattern, stats in effective_ctas[:5]:
            if stats['examples']:
                insights['effective_ctas'].append({
                    'cta_type': self._get_cta_type(pattern),
                    'example': stats['examples'][0][0] if stats['examples'] else '',
                    'avg_score': stats['avg_score'],
                    'usage_count': stats['total_count']
                })

        # Эффективные фразы (униграммы, биграммы)
        for n in [1, 2]:
            if n in self.effective_phrases:
                top_ngrams = sorted(
                    self.effective_phrases[n].items(),
                    key=lambda x: x[1],
                    reverse=True
                )[:5]

                for ngram, count in top_ngrams:
                    insights['content_patterns'].append({
                        'phrase': ' '.join(ngram),
                        'count': count
                    })

        return insights

In [39]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [61]:
class EnhancedTweetEnhancer(TweetEnhancer):
    def __init__(self, historical_df, virality_predictor=None, high_threshold=7.0):
        super().__init__(historical_df, virality_predictor, high_threshold)
        # Популярные эмодзи и их категории
        self.emoji_dict = {
            'positive': ['👍', '😊', '🎉', '✨', '🔥', '💯', '🙌', '❤️', '🚀', '💪'],
            'tech': ['💻', '📱', '🖥️', '☁️', '📊', '🔧', '🌐', '📈', '🤖', '📡'],
            'business': ['💼', '📂', '📈', '💰', '🏢', '🤝', '📊', '📝', '💸', '🔍'],
            'announcement': ['📢', '🔔', '📣', '✅', '⚠️', '📌', '🆕', '🔖', '📰', '🎯']
        }

    def _suggest_emojis(self, text, max_emojis=2):
        """Предлагает эмодзи релевантные тексту твита"""
        # Простая логика для определения подходящих эмодзи
        text_lower = text.lower()
        suggested_emojis = []

        # Анализ текста для выбора категории эмодзи
        if any(word in text_lower for word in ['launch', 'new', 'announce', 'introducing']):
            suggested_emojis.extend(self.emoji_dict['announcement'][:max_emojis])

        if any(word in text_lower for word in ['tech', 'technology', 'computing', 'cloud', 'data', 'digital']):
            suggested_emojis.extend(self.emoji_dict['tech'][:max_emojis])

        if any(word in text_lower for word in ['business', 'company', 'enterprise', 'corporate']):
            suggested_emojis.extend(self.emoji_dict['business'][:max_emojis])

        # Добавляем некоторые позитивные эмодзи по умолчанию, если не выбрали других
        if len(suggested_emojis) < max_emojis:
            suggested_emojis.extend(self.emoji_dict['positive'][:max_emojis - len(suggested_emojis)])

        return suggested_emojis[:max_emojis]

    def enhance_tweet(self, tweet_text, include_time_recommendation=True):
        """Расширенная версия метода с поддержкой эмодзи"""
        # Получаем базовые рекомендации от родительского класса
        recommendations = super().enhance_tweet(tweet_text, include_time_recommendation)

        # Добавляем рекомендации по эмодзи
        recommendations['suggestions']['emojis'] = self._suggest_emojis(tweet_text)

        # Применяем эмодзи к улучшенному твиту
        if 'enhanced_tweet' in recommendations:
            emoji_str = ' ' + ' '.join(recommendations['suggestions']['emojis'])
            recommendations['enhanced_tweet'] += emoji_str

            # Если есть предиктор, обновляем оценку с эмодзи
            if self.virality_predictor:
                current_time = datetime.now()
                weekday = current_time.strftime('%A')
                hour = current_time.hour
                day = current_time.day

                emoji_df = pd.DataFrame({
                    'text': [recommendations['enhanced_tweet']],
                    'normalized_text': [recommendations['enhanced_tweet'].lower()],
                    'Weekday': [weekday],
                    'Hour': [hour],
                    'Day': [day]
                })

                enhanced_predicted = self.virality_predictor.predict(emoji_df)
                recommendations['enhanced_score_with_emoji'] = enhanced_predicted['predicted_score'].iloc[0]
                recommendations['improvement_with_emoji'] = recommendations['enhanced_score_with_emoji'] - recommendations['original_score']

        return recommendations

In [65]:
# Загрузка исторических данных
train_df['normalized_text'] = train_df['normalized_text'].fillna("")
train_df['text'] = train_df['text'].fillna("")
historical_df = train_df

# Загрузка модели предсказания виральности
predictor = TweetViralityPredictor(use_gpu=True)
predictor.load_model("tweet_virality_model.pt")

enhancer = EnhancedTweetEnhancer(historical_df, virality_predictor=predictor)

# Улучшение конкретного твита
tweet = "Current status: Setting up linux server on Amazon Web Services. Next week we'll be negotiating design and rules."
recommendations = enhancer.enhance_tweet(tweet)

# Вывод рекомендаций
print(f"\n\nИсходный твит: {recommendations['original_tweet']}")
print(f"Прогнозируемая виральность: {recommendations['original_score']:.2f}/10")
print(f"\nУлучшенный твит: {recommendations['enhanced_tweet']}")
print(f"Прогнозируемая виральность улучшенного твита: {recommendations['enhanced_score']:.2f}/10")
print(f"\nУлучшенный твит с эмодзи: {recommendations['enhanced_tweet']}")
print(f"Виральность с эмодзи: {recommendations.get('enhanced_score_with_emoji', 0):.2f}/10")
print(f"Улучшение с эмодзи: {recommendations.get('improvement_with_emoji', 0):.2f} пунктов")

# Выводим рекомендации по времени публикации
if 'posting_time' in recommendations:
    print("\nРекомендуемое время публикации:")
    for time_slot in recommendations['posting_time']:
        print(f"- {time_slot['weekday']}, {time_slot['hour']}:00, Средний рейтинг: {time_slot['avg_score']:.2f}")

# Получение общих инсайтов
insights = enhancer.get_content_insights()

Модель загружена из tweet_virality_model.pt


<ipython-input-60-b41280054f0d>:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = self.df['text'].str.contains(pattern, regex=True, na=False)


Получение эмбеддингов...


100%|██████████| 1/1 [00:00<00:00, 29.46it/s]


Получение эмбеддингов...


100%|██████████| 1/1 [00:00<00:00, 32.95it/s]


Получение эмбеддингов...


100%|██████████| 1/1 [00:00<00:00, 31.91it/s]



Исходный твит: Current status: Setting up linux server on Amazon Web Services. Next week we'll be negotiating design and rules.
Прогнозируемая виральность: 2.34/10

Улучшенный твит: Current status: Setting up linux server on Amazon Web Services. Next week we'll be negotiating design and rules. #vic #w #e Learn more 👍 😊
Прогнозируемая виральность улучшенного твита: 2.40/10

Улучшенный твит с эмодзи: Current status: Setting up linux server on Amazon Web Services. Next week we'll be negotiating design and rules. #vic #w #e Learn more 👍 😊
Виральность с эмодзи: 2.38/10
Улучшение с эмодзи: 0.03 пунктов

Рекомендуемое время публикации:
- Saturday, 13:00, Средний рейтинг: 2.82
- Thursday, 16:00, Средний рейтинг: 2.74
- Sunday, 4:00, Средний рейтинг: 2.70
